<h3> Data Cleaning </h3>

In [2]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [5]:
#importing cleaned delegate names
delegates = pd.read_csv("Data/Delegates/cleaned/constitutional_convention_delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)
state_delegates = pd.read_csv("Data/Delegates/cleaned/State_Delegates_cleaned.csv").drop('Unnamed: 0', axis = 1)

# Algorithm for Name Matching

1. Run fuzzy match with score threshold of 80
2. Get unique set of names for each pair, get rid of ???
    1. If one word for either, remove match
    2. Else run word comparison - if there are min(n, m) matches (above 90) then keep match else discard
3. Go through manually

In [7]:
def fuzzy_merge(lst1, lst2, threshold=85, limit = 100):
    """
    :param df_1: the left list to join
    :param df_2: the right list to join
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    
    delegates = pd.Series([x for x in lst1.unique() if not pd.isnull(x)])
    possible =  lst2.unique().tolist()
    
    #get matches
    #process.extract uses a combination of all four fuzzywuzzy scores
    matches = delegates.apply(lambda x: process.extract(x, possible, limit=limit))
    
    match_df = pd.DataFrame(columns = ['Delegates', 'Loan Matches'])
    
    for delegate, matchset in zip(delegates, matches):
        matchset_thres = [name for name in matchset if name[1] >= threshold]
        if len(matchset_thres) == 0:
            add_df = pd.DataFrame(data = {'Delegates': [delegate], 'Loan Matches': [""], 'Scores': [0]})
            match_df = pd.concat([match_df, add_df])
        else:
            delegate_lst = [delegate] * len(matchset_thres)
            add_df = pd.DataFrame(data = {'Delegates': delegate_lst, 
                                          'Loan Matches': [x[0] for x in matchset_thres],
                                          'Scores': [x[1] for x in matchset_thres]})
            match_df = pd.concat([match_df, add_df])

    return match_df

In [8]:
#function for performing the second step of the match
def matchFunction(lst1, lst2, score = 90):
    lst1 = list(set(lst1))
    lst2 = list(set(lst2))
    threshold = min(len(lst1), len(lst2))
    scores = []
    matches = 0
    for wd1 in lst1:
        for wd2 in lst2:
            if process.extract(wd1, [wd2])[0][1] > score:
                matches+=1
                scores.append(fuzz.partial_ratio(wd1, wd2))
    return matches >= threshold

## Matching CT Data

### Matching on Constitutional Convention

In [9]:
def produceMatches(delegates, debt, delegate_names, debt_names):
    initial = True
    join_df = pd.DataFrame()
    for del_name in delegate_names:
        for debt_name in debt_names:
            if initial:
                join_df = fuzzy_merge(delegates[del_name], debt[debt_name])
                initial = False
            else:
                add_df = fuzzy_merge(delegates[del_name], debt[debt_name])
                join_df = pd.concat([join_df, add_df])
    join_df = join_df.drop_duplicates().reset_index(drop = True)
    join_df = join_df[join_df['Scores'].apply(lambda x: x != 0)]
    join_df = join_df[join_df['Loan Matches'].apply(lambda x: not pd.isnull(x))]    
    
    join_df_p2 = join_df[join_df['Loan Matches'].apply(lambda x: len(list(set(x.replace("??", "").strip().split(" "))))>=2)]
    join_df_p2_final = join_df_p2[[matchFunction(x.split(" "), y.split(" ")) for x, y in zip(join_df_p2['Delegates'], join_df_p2['Loan Matches'])]]
    return join_df_p2_final

In [24]:
#prepare loan dataset
CT_ASD = pd.read_excel("Data/Post1790/CT/CT_post1790_ASD_ledger.xlsx", header = 13, usecols = 'H, I, X, Y, AN, AO')
CT_ASD['full name 1'] = CT_ASD['First Name'] + " " + CT_ASD['Last Name']
CT_ASD['full name 2'] = CT_ASD['First Name.1'] + " " + CT_ASD['Last Name.1']
CT_ASD['full name 3'] = CT_ASD['First Name.2'] + " " + CT_ASD['Last Name.2']
CT_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(CT_ASD['full name 1'], CT_ASD['full name 2'])]
CT_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(CT_ASD['full name'], CT_ASD['full name 3'])]

In [25]:
#name matching for constitutional convention and state convention to debt list
CT_ASD_const = produceMatches(delegates, CT_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
CT_ASD_state = produceMatches(state_delegates, CT_ASD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [29]:
#prepare loan dataset
CT_CD = pd.read_excel("Data/Post1790/CT/CT_post1790_CD_ledger.xlsx", header = 13, usecols = 'H, I, X, Y, AN, AO')
CT_CD['full name 1'] = CT_CD['First Name'] + " " + CT_CD['Last Name']
CT_CD['full name 2'] = CT_CD['First Name.1'] + " " + CT_CD['Last Name.1']
CT_CD['full name 3'] = CT_CD['First Name.2'] + " " + CT_CD['Last Name.2']
CT_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(CT_CD['full name 1'], CT_CD['full name 2'])]
CT_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(CT_CD['full name'], CT_CD['full name 3'])]

In [30]:
#name matching for constitutional convention and state convention to debt list
CT_CD_const = produceMatches(delegates, CT_CD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
CT_CD_state = produceMatches(state_delegates, CT_CD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [32]:
#master dataset
cumulative_matching_const = pd.concat([CT_ASD_const, CT_CD_const]).drop_duplicates()
cumulative_matching_state = pd.concat([CT_ASD_state, CT_CD_state]).drop_duplicates()

# Georgia Loan Office Certificates

In [36]:
#prepare pierce certificates
GA = pd.read_excel("Data/Post1790/GA/T694_GA_Loan_Office_CD.xlsx", header = 9, usecols = 'Q, R')

#create full name
GA['full name 1'] = GA['First Name'] + " " + GA['Last Name']

In [38]:
#name matching for constitutional convention and state convention to debt list
GA_const = produceMatches(delegates, GA, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name 1'])
GA_state = produceMatches(state_delegates, GA, delegate_names = ['full name 1'], debt_names = ['full name 1'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [39]:
cumulative_matching_const = pd.concat([cumulative_matching_const, GA_const]).drop_duplicates()

In [40]:
cumulative_matching_state = pd.concat([cumulative_matching_state, GA_state]).drop_duplicates()

# Maryland Debt Certificates

In [64]:
#prepare loan dataset
MD_ASD = pd.read_excel("Data/Post1790/MD/MD_post1790_ASD.xlsx", header = 10, usecols = 'G, H, U, V, AI, AJ')
MD_ASD = MD_ASD.rename(columns={'Unnamed: 34': 'First Name.2', 'Unnamed: 35':'Last Name.2'})
MD_ASD.drop(0, inplace = True)
MD_ASD['full name 1'] = MD_ASD['First Name'] + " " + MD_ASD['Last Name']
MD_ASD['full name 2'] = MD_ASD['First Name.1'] + " " + MD_ASD['Last Name.1']
MD_ASD['full name 3'] = MD_ASD['First Name.2'] + " " + MD_ASD['Last Name.2']
MD_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(MD_ASD['full name 1'], MD_ASD['full name 2'])]
MD_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(MD_ASD['full name'], MD_ASD['full name 3'])]

In [65]:
#name matching for constitutional convention and state convention to debt list
MD_ASD_const = produceMatches(delegates, MD_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
MD_ASD_state = produceMatches(state_delegates, MD_ASD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [74]:
#prepare loan dataset
MD_CD = pd.read_excel("Data/Post1790/MD/MD_post1790_CD.xlsx", header = 10, usecols = 'G, H, U, V, AI, AJ')
MD_CD = MD_CD.rename(columns={'Unnamed: 34': 'First Name.2', 'Unnamed: 35':'Last Name.2'})
MD_CD.drop(0, inplace = True)
MD_CD['full name 1'] = MD_CD['First Name'] + " " + MD_CD['Last Name']
MD_CD['full name 2'] = MD_CD['First Name.1'] + " " + MD_CD['Last Name.1']
MD_CD['full name 3'] = MD_CD['First Name.2'].apply(lambda x: str(x)) + " " + MD_CD['Last Name.2'].apply(lambda x: str(x))
MD_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(MD_CD['full name 1'], MD_CD['full name 2'])]
MD_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(MD_CD['full name'], MD_CD['full name 3'])]

In [75]:
#name matching for constitutional convention and state convention to debt list
MD_CD_const = produceMatches(delegates, MD_CD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
MD_CD_state = produceMatches(state_delegates, MD_CD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [76]:
cumulative_matching_const = pd.concat([cumulative_matching_const,MD_ASD_const, MD_CD_const]).drop_duplicates()

In [77]:
cumulative_matching_state = pd.concat([cumulative_matching_state, MD_ASD_state,MD_CD_state]).drop_duplicates()

# North Carolina

In [79]:
#Read in file
NC_ASD = pd.read_excel("Data/Post1790/NC/T695_R3_NC_ASD.xlsx", header = 9, usecols = 'H, I')

#create full name
NC_ASD['full name 1'] = NC_ASD['First Name'] + " " + NC_ASD['Last Name']

In [80]:
#name matching for constitutional convention and state convention to debt list
NC_ASD_const = produceMatches(delegates, NC_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name 1'])
NC_ASD_state = produceMatches(state_delegates, NC_ASD, delegate_names = ['full name 1'], debt_names = ['full name 1'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [81]:
#Read in file
NC_CD = pd.read_excel("Data/Post1790/NC/T695_R4_NC_CD.xlsx", header = 10, usecols = 'J, K')

#create full name
NC_CD['full name 1'] = NC_CD['First Name'] + " " + NC_CD['Last Name']

In [82]:
#name matching for constitutional convention and state convention to debt list
NC_CD_const = produceMatches(delegates, NC_CD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name 1'])
NC_CD_state = produceMatches(state_delegates, NC_CD, delegate_names = ['full name 1'], debt_names = ['full name 1'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [83]:
cumulative_matching_const = pd.concat([cumulative_matching_const, NC_ASD_const, NC_CD_const]).drop_duplicates()

In [84]:
cumulative_matching_state = pd.concat([cumulative_matching_state, NC_ASD_state, NC_CD_state]).drop_duplicates()

# New Hampshire

In [91]:
#Read in file
NH_ASD = pd.read_excel("Data/Post1790/NH/T652_New_Hampshire_ASD.xlsx", header = 10, usecols = 'G, H, V, W, AK, AL')

#create full name
NH_ASD['full name 1'] = NH_ASD['First Name'] + " " + NH_ASD['Last Name']
NH_ASD['full name 2'] = NH_ASD['First Name.1'] + " " + NH_ASD['Last Name.1']
NH_ASD['full name 3'] = NH_ASD['First Name.2'] + " " + NH_ASD['Last Name.2']
NH_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(NH_ASD['full name 1'], NH_ASD['full name 2'])]
NH_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(NH_ASD['full name'], NH_ASD['full name 3'])]

In [92]:
#name matching for constitutional convention and state convention to debt list
NH_ASD_const = produceMatches(delegates, NH_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
NH_ASD_state = produceMatches(state_delegates, NH_ASD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [93]:
#Read in file
NH_CD = pd.read_excel("Data/Post1790/NH/T652_R6_New_Hampshire_CD.xlsx", header = 10, usecols = 'I, J')

#create full name
NH_CD['full name 1'] = NH_CD['First Name'] + " " + NH_CD['Last Name']

In [94]:
#name matching for constitutional convention and state convention to debt list
NH_CD_const = produceMatches(delegates, NH_CD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name 1'])
NH_CD_state = produceMatches(state_delegates, NH_CD, delegate_names = ['full name 1'], debt_names = ['full name 1'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [95]:
cumulative_matching_const = pd.concat([cumulative_matching_const, NH_ASD_const, NH_CD_const]).drop_duplicates()

In [96]:
cumulative_matching_state = pd.concat([cumulative_matching_state, NH_ASD_state, NH_CD_state]).drop_duplicates()

# New York

In [97]:
#Read in file
NY_ASD = pd.read_excel("Data/Post1790/NY/NY_1790_merged_transfers_removed.xlsx", header = 10, usecols = 'H, I, X, Y, AM, AN')

#create full name
NY_ASD['full name 1'] = NY_ASD['First Name'] + " " + NY_ASD['Last Name']
NY_ASD['full name 2'] = NY_ASD['First Name.1'] + " " + NY_ASD['Last Name.1']
NY_ASD['full name 3'] = NY_ASD['First Name.2'] + " " + NY_ASD['Last Name.2']
NY_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(NY_ASD['full name 1'], NY_ASD['full name 2'])]
NY_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(NY_ASD['full name'], NY_ASD['full name 3'])]

In [98]:
#name matching for constitutional convention and state convention to debt list
NY_ASD_const = produceMatches(delegates, NY_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
NY_ASD_state = produceMatches(state_delegates, NY_ASD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [103]:
cumulative_matching_const = pd.concat([cumulative_matching_const, NY_ASD_const]).drop_duplicates()

In [104]:
cumulative_matching_state = pd.concat([cumulative_matching_state, NY_ASD_state]).drop_duplicates()

# Pennsylvania

In [108]:
#Read in file
PA = pd.read_excel("Data/Post1790/PA/PA_post1790_CD.xlsx", header = 11, usecols = 'G, H, U, V, AJ, AK')

#create full name
PA['full name 1'] = PA['First Name'] + " " + PA['Last Name']
PA['full name 2'] = PA['First Name.1'] + " " + PA['Last Name.1']
PA['full name 3'] = PA['First Name.2'] + " " + PA['Last Name.2']
PA['full name'] = [y if pd.isnull(x) else x for x, y in zip(PA['full name 1'], PA['full name 2'])]
PA['full name'] = [y if pd.isnull(x) else x for x, y in zip(PA['full name'], PA['full name 3'])]

In [110]:
#name matching for constitutional convention and state convention to debt list
PA_const = produceMatches(delegates, PA, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
PA_state = produceMatches(state_delegates, PA, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [111]:
cumulative_matching_const = pd.concat([cumulative_matching_const, PA_const]).drop_duplicates()

In [112]:
cumulative_matching_state = pd.concat([cumulative_matching_state, PA_state]).drop_duplicates()

# Rhode Island

In [118]:
#Read in file
RI_ASD = pd.read_excel("Data/Post1790/RI/T653_Rhode_Island_ASD.xlsx", header = 10, usecols = 'H, I, X, Y, AN, AO')

#create full name
RI_ASD['full name 1'] = RI_ASD['First Name'] + " " + RI_ASD['Last Name']
RI_ASD['full name 2'] = RI_ASD['First Name.1'] + " " + RI_ASD['Last Name.1']
RI_ASD['full name 3'] = RI_ASD['First Name.2'] + " " + RI_ASD['Last Name.2']
RI_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(RI_ASD['full name 1'], RI_ASD['full name 2'])]
RI_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(RI_ASD['full name'], RI_ASD['full name 3'])]

In [119]:
#name matching for constitutional convention and state convention to debt list
RI_ASD_const = produceMatches(delegates, RI_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
RI_ASD_state = produceMatches(state_delegates, RI_ASD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [120]:
#Read in file
RI_CD = pd.read_excel("Data/Post1790/RI/T653_Rhode_Island_CD.xlsx", header = 10, usecols = 'G, H, U, V, AI, AJ')

#create full name
RI_CD['full name 1'] = RI_CD['First Name'] + " " + RI_CD['Last Name']
RI_CD['full name 2'] = RI_CD['First Name.1'] + " " + RI_CD['Last Name.1']
RI_CD['full name 3'] = RI_CD['First Name.2'] + " " + RI_CD['Last Name.2']
RI_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(RI_CD['full name 1'], RI_CD['full name 2'])]
RI_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(RI_CD['full name'], RI_CD['full name 3'])]

In [121]:
#name matching for constitutional convention and state convention to debt list
RI_CD_const = produceMatches(delegates, RI_CD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
RI_CD_state = produceMatches(state_delegates, RI_CD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [122]:
cumulative_matching_const = pd.concat([cumulative_matching_const, RI_ASD_const, RI_CD_const]).drop_duplicates()

In [123]:
cumulative_matching_state = pd.concat([cumulative_matching_state, RI_ASD_state, RI_CD_state]).drop_duplicates()

# South Carolina

In [126]:
#Read in file
SC = pd.read_excel("Data/Post1790/SC/Post_1790_South_Carolina_consolidated.xlsx", header = 11, usecols = 'D, E, S, T, AH, AI')

#create full name
SC['full name 1'] = SC['First Name'] + " " + SC['Last Name']
SC['full name 2'] = SC['First Name.1'] + " " + SC['Last Name.1']
SC['full name 3'] = SC['First Name.2'] + " " + SC['Last Name.2']
SC['full name'] = [y if pd.isnull(x) else x for x, y in zip(SC['full name 1'], SC['full name 2'])]
SC['full name'] = [y if pd.isnull(x) else x for x, y in zip(SC['full name'], SC['full name 3'])]

In [127]:
#name matching for constitutional convention and state convention to debt list
SC_const = produceMatches(delegates, SC, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
SC_state = produceMatches(state_delegates, SC, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [129]:
cumulative_matching_const = pd.concat([cumulative_matching_const, SC_const]).drop_duplicates()

In [128]:
cumulative_matching_state = pd.concat([cumulative_matching_state, SC_state]).drop_duplicates()

# Virginia

In [137]:
#Read in file
VA_ASD = pd.read_excel("Data/Post1790/VA/VA_ASD.xlsx", header = 10, usecols = 'D, E, U, V, AL, AM')
VA_ASD.rename(columns = {'Unnamed: 3':'First Name.0', 'Unnamed: 4':'Last Name.0'}, inplace = True)
VA_ASD.drop(0, inplace = True)
#create full name
VA_ASD['full name 1'] = VA_ASD['First Name.0'] + " " + VA_ASD['Last Name.0']
VA_ASD['full name 2'] = VA_ASD['First Name'] + " " + VA_ASD['Last Name']
VA_ASD['full name 3'] = VA_ASD['First Name.1'] + " " + VA_ASD['Last Name.1']
VA_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(VA_ASD['full name 1'], VA_ASD['full name 2'])]
VA_ASD['full name'] = [y if pd.isnull(x) else x for x, y in zip(VA_ASD['full name'], VA_ASD['full name 3'])]

In [138]:
#name matching for constitutional convention and state convention to debt list
VA_ASD_const = produceMatches(delegates, VA_ASD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
VA_ASD_state = produceMatches(state_delegates, VA_ASD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [144]:
#Read in file
VA_CD = pd.read_excel("Data/Post1790/VA/VA_CD.xlsx", header = 11, usecols = 'H, I, U, V, AH, AI')
VA_CD.rename(columns = {'Unnamed: 7':'First Name.0', 'Unnamed: 8':'Last Name.0'}, inplace = True)
VA_CD.drop(0, inplace = True)
#create full name
VA_CD['full name 1'] = VA_CD['First Name.0'] + " " + VA_CD['Last Name.0']
VA_CD['full name 2'] = VA_CD['First Name'] + " " + VA_CD['Last Name']
VA_CD['full name 3'] = VA_CD['First Name.1'] + " " + VA_CD['Last Name.1']
VA_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(VA_CD['full name 1'], VA_CD['full name 2'])]
VA_CD['full name'] = [y if pd.isnull(x) else x for x, y in zip(VA_CD['full name'], VA_CD['full name 3'])]

In [145]:
#name matching for constitutional convention and state convention to debt list
VA_CD_const = produceMatches(delegates, VA_CD, delegate_names = ['full name 1', 'full name 2'], debt_names = ['full name'])
VA_CD_state = produceMatches(state_delegates, VA_CD, delegate_names = ['full name 1'], debt_names = ['full name'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [146]:
cumulative_matching_const = pd.concat([cumulative_matching_const, VA_ASD_const, VA_CD_const]).drop_duplicates()

In [147]:
cumulative_matching_state = pd.concat([cumulative_matching_state, VA_ASD_state, VA_CD_state]).drop_duplicates()

# Final Steps

In [155]:
cum_matching = pd.concat([cumulative_matching_const, cumulative_matching_state]).drop_duplicates()

In [156]:
cum_matching.drop(cum_matching[[x and y<88 for x, y in zip(cum_matching['Loan Matches'].duplicated(), cum_matching['Scores'])]].index, inplace = True)

In [158]:
cum_matching = cum_matching[cum_matching['Scores'] > 85]

In [159]:
cum_matching = cum_matching.sort_values(['Delegates', 'Loan Matches', 'Scores'])

In [162]:
rem_index = list()

In [163]:
rem_index.extend(cum_matching[[x.lower() == 'john thomas' and y < 90 for x, y in zip(cum_matching['Delegates'], cum_matching['Scores'])]].index)

In [164]:
rem_index.extend(cum_matching[[x.lower() == 'william williams' and y < 90 for x, y in zip(cum_matching['Delegates'], cum_matching['Scores'])]].index)

In [165]:
cum_matching.drop(rem_index, inplace = True)

In [166]:
cum_matching.to_csv('Data/final_matching_post1790.csv')